# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import TransformerMixin,BaseEstimator
from nltk.tokenize import word_tokenize,sent_tokenize

from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
import pickle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pickle
from nltk.stem.porter import PorterStemmer

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///disasterResponse.db')
df = pd.read_sql('SELECT * FROM message_categories',con=engine,index_col='id')
df.head() 

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#check unique values in each column
df.nunique()

message                   26177
original                   9630
genre                         3
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
child_alone                   1
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_infrastructure          2
weather_

Since child_alone has a single value, this would not help in the learning. We need to drop this column. Additionally, related column has 3 unique values. 

In [5]:
df.related.value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

There are 0,1,2 values in related column. It looks like value 2 is an error in related column. We can drop these 188 rows or impute it with 0 or 1. In this case, we will impute it with mode value of 1.

In [6]:
#drop child_alone column
df = df.drop(columns=['child_alone'])

In [7]:
#Replace 2 with 1 in related column 
df['related'] = df.related.apply(lambda x: 1 if x == 2 else x)

In [8]:
#TEST
df.related.value_counts()

1    20094
0     6122
Name: related, dtype: int64

In [9]:
#Predict the boolean varaibles from message column 

#select message column
X = df['message'].dropna()

#skip first three columns : message, original, genre and select the rest of the columns
Y = df.iloc[:,3:].dropna()

In [10]:
#TEST: Count number of columns to predict
Y.shape

(26216, 35)

In [11]:
#check for null values
df.isna().sum()

message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_infrastructure          0
weather_related               0
floods  

Only original has null values. This does not affect our prediction, so we will ignore it.

### 2. Write a tokenization function to process your text data

In [12]:
def tokenize(text):
    """
    Tokenize the text
    
    Input:
        text -> Text message to tokenize
    Output:
        clean_tokens -> List of tokens extracted from text
    """
    
    #declare url regex to detect a url
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'  

    
    #replace the url with text : urlplaceholder
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    #normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())      
   
    #tokenize text
    #tokens = word_tokenize(text)
    tokens = word_tokenize(text)    
    
    
    
    # stopword list 
    stop_words = stopwords.words("english")
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    #lemmatize text
    lemmatizer = WordNetLemmatizer()
    
   
    
    #generate a list of clean tokens 
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [13]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    A class used to extract starting verb of the text to generate a new feature 
    Class inherited both BaseEstimator and TransformerMixin classes of sklearn.base
    
    Methods:
    --------
        starting_verb(self,text) -> Returns 1 if first word of text is a verb/RT else 0 
        fit(self,x,y) -> Returns self 
        transform(self,X) -> Applies starting_verb function to all values of X

    """

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = pos_tag(tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return 1

            return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline = Pipeline([                 
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),              
                ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])           

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
#split data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [16]:
#fit the model
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred_train = pipeline_fitted.predict(X_train)
y_pred_test = pipeline_fitted.predict(X_test)

#classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      4980
               request       0.78      0.44      0.56      1067
                 offer       0.00      0.00      0.00        30
           aid_related       0.74      0.59      0.66      2648
          medical_help       0.57      0.13      0.21       525
      medical_products       0.78      0.11      0.20       312
     search_and_rescue       0.42      0.03      0.05       178
              security       0.00      0.00      0.00       113
              military       0.66      0.10      0.17       189
                 water       0.88      0.31      0.46       397
                  food       0.83      0.45      0.58       708
               shelter       0.83      0.33      0.47       568
              clothing       0.73      0.12      0.21        91
                 money       0.75      0.04      0.07       152
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'classifier', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'classifier__estimator__bootstrap', 'classifier__estimator__class_weight', 'classifier__estimator__criterion', 'classifier__estimator__max_depth', 'classifier__estimator__max_features', 'classifier__estimator__max_leaf_nodes', 'classifier__estimator__min_impurity_decrease', 'classifier__estimator__min_impurity_split', 'classifier__estimator__min_samples_leaf', 'classifier__estimator__min_samples_split', 'classifier__estimator__min_weight_fraction_leaf', 'classifier__estimator__n_estimators', 'classifier__estimator__n_jobs', 'classi

In [19]:
#select parameters for grid search
#parameters = {
        #'features__nlp_pipeline__vect__ngram_range':((1, 1), (1, 2)),
        #'tfidf__use_idf': (True, False)
        #,'classifier__estimator__n_estimators': [10, 20, 40],
        #,'features__nlp_pipeline__vect__max_features': (None, 5000, 10000),
        #'vect__max_df':(0.5, 0.75, 1.0)
#        'classifier__estimator__learning_rate': [0.01, 0.02, 0.05]
        #'features__nlp_pipeline__vect__min_df': [2, 4]
    #}

parameters = {'classifier__estimator__max_depth': [2,5, None],
              'classifier__estimator__min_samples_leaf':[2, 5 , None]}

# call gridsearch on above parameters. And set cv=2 and n_jobs=-1 to speed up the process
cv = GridSearchCV(pipeline, param_grid=parameters, cv = 3, n_jobs=-1,verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
cv.fit(X_train,y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=2 
[CV]  classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=2, score=0.20231919438510834, total= 1.7min
[CV] classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=2 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.8min remaining:    0.0s


[CV]  classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=2, score=0.20125114433933475, total= 1.7min
[CV] classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=2 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  5.6min remaining:    0.0s


[CV]  classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=2, score=0.20125114433933475, total= 1.7min
[CV] classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=5 


...

[CV]  classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=5, score=0.20109856576136712, total= 1.7min
[CV] classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=5 


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 13.9min remaining:    0.0s


[CV]  classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=5, score=0.2007934086054318, total= 1.7min
[CV] classifier__estimator__max_depth=2, classifier__estimator__min_samples_leaf=None 


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed: 16.6min remaining:    0.0s


TypeError: '<' not supported between instances of 'float' and 'NoneType'

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])

num=0

for cat in Y.columns:
    precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,num], average='weighted')    
    results.loc[num] = [cat,f_score, precision, recall]    
    num = num + 1

print('Mean f_score:', results['f_score'].mean())
print('Mean precision:', results['precision'].mean())
print('Mean recall:', results['recall'].mean())


In [ ]:
results.head()

In [ ]:
#find the best combination and values of parameters
cv.best_params_

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
parameters = {'classifier__estimator__max_depth': [10, 50, None],
              'classifier__estimator__min_samples_leaf':[2, 5, 10],
              'tfidf__use_idf': (True, False)}

# call gridsearch on above parameters. And set cv=2 and n_jobs=-1 to speed up the process
cv = GridSearchCV(pipeline, param_grid=parameters, cv = 3, n_jobs=-1)

In [ ]:
cv.fit(X_train,y_train)

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])

num=0

for cat in Y.columns:
    precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,num], average='weighted')
    results.loc[num] = [cat,f_score, precision, recall]    
    num = num + 1

print('Mean f_score:', results['f_score'].mean())
print('Mean precision:', results['precision'].mean())
print('AMean recall:', results['recall'].mean())


### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv.best_estimator_, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.